In [4]:
import os
from simulate_data import simulate_data
from cassiopeia.data import CassiopeiaTree
from cassiopeia.solver import VanillaGreedySolver
from cassiopeia.critique import triplets_correct
import pickle
import numpy as np
from ..heracles2.metrics import cas_triplets_correct
import torch

# automatically reload modules
%load_ext autoreload
%autoreload 2

ImportError: attempted relative import with no known parent package

In [ ]:
num_states=15
num_sites=5
mutation_rate=0.4
deletion_rate=9e-4    
transition_prob = {i: 1/num_states for i in range(num_states)}
path = '../heracles2/data'

simulate_data(transition_prob, num_sites, num_states, mutation_rate, deletion_rate, path)

In [ ]:
fname = os.path.join(path, 'true_tree')
with open(fname, 'rb') as file:
    true_tree = pickle.load(file)
    
cas_tree = CassiopeiaTree(character_matrix=true_tree.character_matrix, priors=None)
vanilla_greedy = VanillaGreedySolver()
vanilla_greedy.solve(cas_tree, collapse_mutationless_edges=True)
triplets = triplets_correct(true_tree, cas_tree)
cas_ans = np.mean(list(triplets[0].values()))

In [ ]:
os.system('python ../heracles2/main.py --tree_path ../heracles2/data/true_tree --params_path ../heracles2/data/params --save_path ../heracles2/saved')
rho = torch.tensor(2, dtype=torch.float64)

# read in best embeddings X from file
X = torch.load('../heracles2/saved/best_embeddings.pt')
heracles_ans = cas_triplets_correct(true_tree, X, rho)

In [ ]:
print(cas_ans, heracles_ans)